# Tutorial - build LinearSVC model with sklearn

This tutorial demonstrates how to use the Sci-kit Learn (sklearn) package to build linearSVC model, rank features, and use the model for prediction. We will be using the Kaggle sentiment data again.

Note that sklearn actually provides two SVM algorithms: SVC and LinearSVC. 

The SVC module allows for choosing nonlinear kernels, and it uses one-vs-one strategy for multi-class classification.

The LinearSVC module uses the linear kernel, and it uses one-vs-all strategy for multi-class classification, so linearSVC is generally faster than SVC. Since linear kernel works better for text classification in general, this tutorial demonstrates how to use LinearSVC for text classification.

# Step 1: Read in data

In [1]:
# this step is the same as the NB script

# read in the training data
# the data set includes four columns: PhraseId, SentenceId, Phrase, Sentiment
# In this data set a sentence is further split into phrases 
# in order to build a sentiment classification model
# that can not only predict sentiment of sentences but also shorter phrases

# A data example:
# PhraseId SentenceId Phrase Sentiment
# 1 1 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .1

# the Phrase column includes the training examples
# the Sentiment column includes the training labels
# "0" for very negative
# "1" for negative
# "2" for neutral
# "3" for positive
# "4" for very positive


import pandas as p
import numpy as np
train=p.read_csv("/Users/shivangi/Downloads/kaggle-sentiment/train.tsv", delimiter='\t')
y=train['Sentiment'].values
X=train['Phrase'].values

# Step 2: Split train/test data for hold-out test

In [2]:
# this step is the same as the NB script

# check the sklearn documentation for train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# "test_size" : float, int, None, optional
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
# If int, represents the absolute number of test samples. 
# If None, the value is set to the complement of the train size. 
# By default, the value is set to 0.25. The default will change in version 0.21. It will remain 0.25 only if train_size is unspecified, otherwise it will complement the specified train_size.    

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2


Sample output from the code above:

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2

# Step 2.1 Data Checking

In [3]:
# this step is the same as the NB script

# Check how many training examples in each category
# this is important to see whether the data set is balanced or skewed

unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))

[[    0     1     2     3     4]
 [ 4141 16449 47718 19859  5469]]


The sample output shows that the data set is skewed with 47718/93636=51% "neutral" examples. All other categories are smaller.

{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]

# Step 3: Vectorization

In [4]:
# this step is the same as the NB script

# sklearn contains two vectorizers

# CountVectorizer can give you Boolean or TF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# TfidfVectorizer can give you TF or TFIDF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Read the sklearn documentation to understand all vectorization options

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# several commonly used vectorizer setting

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')

#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')

#  unigram tfidf vectorizer, set minimum document frequency to 5
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')


## Step 3.1: Vectorize the training data

In [5]:
# this step is the same as the NB script

# The vectorizer can do "fit" and "transform"
# fit is a process to collect unique tokens into the vocabulary
# transform is a process to convert each document to vector based on the vocabulary
# These two processes can be done together using fit_transform(), or used individually: fit() or transform()

# fit vocabulary in training documents and transform the training documents into vectors
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)

# check the content of a document vector
print(X_train_vec.shape)
print(X_train_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(unigram_count_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(unigram_count_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(unigram_count_vectorizer.vocabulary_.get('imaginative'))

(93636, 11967)
[[0 0 0 ... 0 0 0]]
11967
[('class', 1858), ('wilde', 11742), ('derring', 2802), ('chilling', 1764), ('affecting', 313), ('meanspirited', 6557), ('personal', 7662), ('low', 6296), ('involved', 5602), ('worth', 11868)]
5224


Sample output:

(93636, 11967)
[[0 0 0 ..., 0 0 0]]
11967
[('imaginative', 5224), ('tom', 10809), ('smiling', 9708), ('easy', 3310), ('diversity', 3060), ('impossibly', 5279), ('buy', 1458), ('sentiments', 9305), ('households', 5095), ('deteriorates', 2843)]
5224

## Step 3.2: Vectorize the test data

In [6]:
# this step is the same as the NB script

# use the vocabulary constructed from the training data to vectorize the test data. 
# Therefore, use "transform" only, not "fit_transform", 
# otherwise "fit" would generate a new vocabulary from the test data

X_test_vec = unigram_count_vectorizer.transform(X_test)

# print out #examples and #features in the test set
print(X_test_vec.shape)

(62424, 11967)


Sample output:

(62424, 14324)

# Step 4: Train a LinearSVC classifier

In [7]:
# import the LinearSVC module
from sklearn.svm import LinearSVC

# initialize the LinearSVC model
svm_clf = LinearSVC(C=1)

# use the training data to train the model
svm_clf.fit(X_train_vec,y_train)

LinearSVC(C=1)

# Step 4.1 Interpret a trained LinearSVC model

In [8]:
## interpreting LinearSVC models
## http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

## LinearSVC uses a one-vs-all strategy to extend the binary SVM classifier to multi-class problems
## for the Kaggle sentiment classification problem, there are five categories 0,1,2,3,4 with 0 as very negative and 4 very positive
## LinearSVC builds five binary classifier, "very negative vs. others", "negative vs. others", "neutral vs. others", "positive vs. others", "very positive vs. others", 
## and then pick the most confident prediction as the final prediction.

## Linear SVC also ranks all features based on their contribution to distinguish the two concepts in each binary classifier
## For category "0" (very negative), get all features and their weights and sort them in increasing order
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names_out()))

## get the 10 features that are best indicators of very negative sentiment (they are at the bottom of the ranked list)
very_negative_10 = feature_ranks[-10:]
print("Very negative words")
for i in range(0, len(very_negative_10)):
    print(very_negative_10[i])
print()

## get 10 features that are least relevant to "very negative" sentiment (they are at the top of the ranked list)
not_very_negative_10 = feature_ranks[:10]
print("not very negative words")
for i in range(0, len(not_very_negative_10)):
    print(not_very_negative_10[i])
print()


Very negative words
(1.6216100478988977, 'cesspool')
(1.6484881167517755, 'disappointment')
(1.6592494849306287, 'pompous')
(1.6683696601864677, 'stinks')
(1.6927739819447036, 'distasteful')
(1.6955905277536816, 'unwatchable')
(1.7526396850091917, 'unbearable')
(1.7873567099435386, 'stinker')
(1.8228706148331673, 'disgusting')
(1.8233056914034858, 'worthless')

not very negative words
(-1.8329269730191917, 'hawke')
(-1.7372807482923114, 'giddy')
(-1.6832952952268576, 'collar')
(-1.5847293714382105, 'swimfan')
(-1.572076515413363, 'blue')
(-1.4801113883885788, 'dogtown')
(-1.413836129011043, 'clamoring')
(-1.4093532373501032, 'joan')
(-1.3918161535273166, 'victim')
(-1.340000173886529, 'compulsively')



Sample output:

Very negative words
(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

not very negative words
(-2.3147454187985117, 'sacrifices')
(-1.8650987318574794, 'maintained')
(-1.8305667747223913, 'placed')
(-1.7974037295239631, 'argue')
(-1.6800998534753624, '19')
(-1.6684863939524339, 'homage')
(-1.6179084517399509, 'failure')
(-1.6088792786048403, 'breezy')
(-1.6059138072144292, 'bore')
(-1.5466693614369267, 'clone')

# Exercise A

In [9]:
# write code similar to the above sample code 
# and print out the 10 most indicative words for the "very positive" category and the "positive" category

# Your code starts here

feature_ranks_vpos = sorted(zip(svm_clf.coef_[4], unigram_count_vectorizer.get_feature_names_out()))
feature_ranks_pos = sorted(zip(svm_clf.coef_[3], unigram_count_vectorizer.get_feature_names_out()))

## get the 10 features that are best indicators of very positive sentiment (they are at the bottom of the ranked list)
very_positive_10 = feature_ranks_vpos[-10:]
print("Very positive words")
for i in range(0, len(very_positive_10)):
    print(very_positive_10[i])
print()

## get the 10 features that are best indicators of very positive sentiment (they are at the bottom of the ranked list)
positive_10 = feature_ranks_pos[-10:]
print("Positive words")
for i in range(0, len(positive_10)):
    print(positive_10[i])
print()

# Your code ends here

Very positive words
(1.563528620613832, 'stunning')
(1.6005794341085797, 'astonish')
(1.6108129517559653, 'refreshes')
(1.6148905709992567, 'flawless')
(1.647464686366039, 'phenomenal')
(1.6506426771239329, 'masterful')
(1.677615581239628, 'masterfully')
(1.8781421165591192, 'glorious')
(1.9801882958590709, 'miraculous')
(2.0143252330058212, 'perfection')

Positive words
(1.6394010901109861, 'embraced')
(1.6458101037476704, 'clenching')
(1.6782541137342109, 'pulpiness')
(1.6967191349252824, 'companionable')
(1.713004861933028, 'cheerfully')
(1.87989344435394, 'efficient')
(1.8853119829860951, 'knotting')
(1.9785036060574466, 'irresistibly')
(1.9907525876504681, 'ideally')
(2.111387447241931, 'marveled')



Sample output

(1.5635285973159958, 'stunning')
(1.6005795078256047, 'astonish')
(1.610812924790558, 'refreshes')
(1.6148905161082145, 'flawless')
(1.6474647033330083, 'phenomenal')
(1.6506425169734038, 'masterful')
(1.67761558779458, 'masterfully')
(1.8781421016763864, 'glorious')
(1.9801881772566481, 'miraculous')
(2.0143251933052397, 'perfection')

# Step 5: Test the LinearSVC classifier

In [10]:
# test the classifier on the test data set, print accuracy score

svm_clf.score(X_test_vec,y_test)



0.6236864026656415

In [11]:
# print confusion matrix and classification report

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4])
print(cm)
print()

from sklearn.metrics import classification_report
target_names = ['0','1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[[  918  1221   697    82    13]
 [  701  4080  5504   514    25]
 [  195  2106 27081  2310   172]
 [   34   396  6048  5533  1057]
 [    3    51   590  1772  1321]]

              precision    recall  f1-score   support

           0       0.50      0.31      0.38      2931
           1       0.52      0.38      0.44     10824
           2       0.68      0.85      0.75     31864
           3       0.54      0.42      0.48     13068
           4       0.51      0.35      0.42      3737

    accuracy                           0.62     62424
   macro avg       0.55      0.46      0.49     62424
weighted avg       0.60      0.62      0.60     62424



# Step 5.1 Interpret the prediction result

In [12]:
## get the confidence scores for all test examples from each of the five binary classifiers
svm_confidence_scores = svm_clf.decision_function(X_test_vec)
## get the confidence score for the first test example
print(svm_confidence_scores[0])

## sample output: array([-1.05306321, -0.62746206,  0.31074854, -0.89709483, -1.08343089]
## because the confidence score is the highest for category 2, 
## the prediction should be 2. 

## Confirm by printing out the actual prediction
print(y_test[0])
print(X[0])

[-1.01718388 -0.50760034  0.22331219 -0.97514735 -1.24718847]
2
A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


In [13]:
# output prediction probs
# https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py

from sklearn.calibration import CalibratedClassifierCV
svm_calibrated = CalibratedClassifierCV(svm_clf)
svm_calibrated.fit(X_train_vec, y_train)
y_test_proba = svm_calibrated.predict_proba(X_test_vec)
y_test_proba[0]

/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([0.03909122, 0.20406247, 0.60822951, 0.11914356, 0.02947324])

# Exercise B

In [14]:
# The above sample code examines LinearSVC's prediction confidence for the first example.
# Now you are going to print out another example with index=99 (the 100th example), 
# and examine the LinearSVC's prediction confidence and its final prediction
# does this prediction seem correct to you?

# Your code starts here
print(svm_confidence_scores[99])
## Confirm by printing out the actual prediction
print(y_test[99])
print(X[99])
# output prediction probs
y_test_proba[99]
# Your code ends here

[-1.18044093 -0.55340772  0.22562433 -0.62179447 -2.11601384]
2
suspect


array([0.02923082, 0.1907494 , 0.59051513, 0.18406636, 0.00543828])

The prediction seems correct as the highest value of confidence score is for category 2 and the actual is 2.

# Step 5.2 Error Analysis

In [15]:
# print out specific type of error for further analysis

# print out the very positive examples that are mistakenly predicted as negative
# according to the confusion matrix, there should be 53 such examples
# note if you use a different vectorizer option, your result might be different

err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

10 minutes into the film you 'll be white-knuckled and unable to look away .
the film is never dull
greatly impressed by the skill of the actors involved in the enterprise
errors: 3


# Exercise C

In [16]:
# write code to print out 
# the errors that very negative examples were mistakenly predicted as very positive?
# and the errors that very positive examples were mistakenly predicted as very negative?

# Try find lingustic patterns for these two types of errors
# Based on the above error analysis, what suggestions would you give to improve the current model?

# Your code starts here
print("very negative examples were mistakenly predicted as very positive - ")
err_cntC = 0
for i in range(0, len(y_test)):
    if(y_test[i]==0 and y_pred[i]==4):
        print(X_test[i])
        err_cntC = err_cntC+1
print("errors:", err_cntC)

print("very positive examples were mistakenly predicted as very negative - ")
err_cntC1 = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cntC1 = err_cntC1+1
print("errors:", err_cntC1)
# Your code ends here

very negative examples were mistakenly predicted as very positive - 
the film is not entirely successful
manages to bleed it almost completely dry of humor , verve and fun .
the spell they cast is n't the least bit mesmerizing
The Master of Disguise is funny -- not
being cast in action films when none of them are ever any good
But if the essence of magic is its make-believe promise of life that soars above the material realm , this is the opposite of a truly magical movie .
is not Edward Burns ' best film
bleed it almost completely dry of humor , verve and fun
Once the expectation of laughter has been quashed by whatever obscenity is at hand , even the funniest idea is n't funny .
manages to bleed it almost completely dry of humor , verve and fun
It may not be a great piece of filmmaking ,
Depicts the sorriest and most sordid of human behavior on the screen , then laughs at how clever it 's being .
a great movie it is not
errors: 13
very positive examples were mistakenly predicted as v

# Step 6. Cross Validation

In [17]:
# Cross validation
# for more details see https://scikit-learn.org/stable/modules/cross_validation.html
# evaluation metrics provided by sklearn - https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

# if you need to output only one metric, such as accuracy or f1_macro, use the "cross_val_score" function

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=1))])
scores = cross_val_score(svm_clf_pipe, X, y, cv=3, scoring='f1_macro')

/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [18]:
# if you need to output multiple metrics, such as accuracy and f1_macro, use the "cross_validate" function
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'f1_macro', 'f1_micro', 'precision_macro', 'recall_macro']
svm_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=1))])
scores = cross_validate(svm_clf_pipe, X, y, cv=3, scoring=scoring, return_train_score=True)

/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_accuracy',
 'test_f1_macro',
 'test_f1_micro',
 'test_precision_macro',
 'test_recall_macro',
 'train_accuracy',
 'train_f1_macro',
 'train_f1_micro',
 'train_precision_macro',
 'train_recall_macro']

In [20]:
# retrieve scores from a metric
def get_metric_scores (scores, metric, train_or_test, verbose=False):
    metric_name = train_or_test + '_' + metric
    print(metric_name) 

    metric_scores = scores[metric_name]
    if (verbose == True):
        print(metric_scores)
    avg = sum(metric_scores) / len(metric_scores)
    print('average')
    avg_formatted = "{:.3f}".format(avg)
    print(avg_formatted)

In [21]:
#retrieve test accuracy scores
get_metric_scores(scores, 'accuracy', 'test', verbose=True)

test_accuracy
[0.57604767 0.56403306 0.56680123]
average
0.569


In [22]:
#retrieve training accuracy scores
get_metric_scores(scores, 'accuracy', 'train')

train_accuracy
average
0.738


In [23]:
# compare performance with different choice of C values

svm_clf_pipe2 = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=0.5))])
scores2 = cross_validate(svm_clf_pipe2, X, y, cv=3, scoring=scoring, return_train_score=True)

svm_clf_pipe3 = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('svm', LinearSVC(C=2))])
scores3 = cross_validate(svm_clf_pipe3, X, y, cv=3, scoring=scoring, return_train_score=True)

/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/shivangi/opt/anaconda3/lib/python3.9/site-packages/sklearn

In [24]:
# compare the effect of different C values
# C=1
print('C=1\n')
get_metric_scores(scores, 'accuracy', 'train')
get_metric_scores(scores, 'accuracy', 'test')

# C=0.5
print('\nC=0.5\n')
get_metric_scores(scores2, 'accuracy', 'train')
get_metric_scores(scores2, 'accuracy', 'test')

# C=2
print('\nC=2\n')
get_metric_scores(scores3, 'accuracy', 'train')
get_metric_scores(scores3, 'accuracy', 'test')



C=1

train_accuracy
average
0.738
test_accuracy
average
0.569

C=0.5

train_accuracy
average
0.732
test_accuracy
average
0.574

C=2

train_accuracy
average
0.741
test_accuracy
average
0.565


# Exercise D

In [25]:
# retrieve the f1_macro scores and compare the effect of C on f1_macro 

# Your code starts here
# C=1
print('C=1\n')
get_metric_scores(scores, 'f1_macro', 'train')
get_metric_scores(scores, 'f1_macro', 'test')

# C=0.5
print('\nC=0.5\n')
get_metric_scores(scores2, 'f1_macro', 'train')
get_metric_scores(scores2, 'f1_macro', 'test')

# C=2
print('\nC=2\n')
get_metric_scores(scores3, 'f1_macro', 'train')
get_metric_scores(scores3, 'f1_macro', 'test')

# Your code ends here

C=1

train_f1_macro
average
0.665
test_f1_macro
average
0.409

C=0.5

train_f1_macro
average
0.655
test_f1_macro
average
0.409

C=2

train_f1_macro
average
0.673
test_f1_macro
average
0.408
